## Archivo de limpieza y preparacion para los mapas

### 1_ Preparamos el entorno

In [1]:
import re
import warnings

import pandas as pd

warnings.filterwarnings('ignore')
import unicodedata

____ ACA CAMBIAR EL ARCHIVO AL NUEVO QUE SE QUIERA MAPEAR ___ 


In [2]:
# JULIO
#track = pd.read_excel("Reporte de viaje(20240701-20240731).xlsx", engine="openpyxl")

# AGOSTO
track = pd.read_excel("Reporte de viaje(20240801-20240831).xlsx", engine="openpyxl")

**Cargamos ubicaciones extra Abitab,Red pagos, Empresa, Casas**

In [3]:
lugares_pago = {
        'CODIGO': [200000, 200002, 200003, 200005, 200006, 200007, 200008, 200009, 200010, 200011, 200012, 200013, 200014, 200015, 200016, 200017, 200018, 200019, 200020, 200021, 200022, 200023, 200024, 200025, 200026, 200027, 200028, 200029, 200030, 200031],
        'NOMCLI': ['Abitab', 'ABITAB 12/18', 'Abitab', 'Abitab Ruta 87', 'Abitab 12-21', 'Abitab', 'Abitab', 'Cambilex', 'Abitab', 'Abitab', 'Abitab', 'Abitab', 'Abitab', 'Abitab', 'Abitab Agencia 18/09', 'Redpagos Minimercado Nico', 'Red Pagos El Dorado, Las Toscas', 'Redpagos Bachino', 'Redpagos San Luis', 'Redpagos La Tuna', 'Redpagos Provision San Jorge', 'Red Pagos', 'RedPagos', 'RedPagos', 'Cambilex', 'Redpagos Salinas', 'Redpagos Salinas Norte', 'Redpagos Pinamar', 'Redpagos Agencias Migues', 'Redpagos Saucedo'],
        'LATITUD': [-34.7773749, -34.778762, -34.7769439, -34.7722784, -34.7717067, -34.7576624, -34.7714756, -34.772715, -34.7554596, -34.7673311, -34.7506218, -34.758673, -34.769283, -34.7701505, -34.6003052, -34.79286, -34.7710358, -34.7592078, -34.7663154, -34.7801148, -34.6832898, -34.7499177, -34.7741445, -34.7679195, -34.7773725, -34.7758541, -34.77187, -34.778569, -34.4884583, -34.5999389],
        'LONGITUD': [-55.8391627, -55.8620459, -55.8489382, -55.8413815, -55.8195923, -55.766793, -55.7620991, -55.7637416, -55.7246934, -55.7220688, -55.7094748, -55.677871, -55.6553415, -55.5763933, -55.4663981, -55.49211, -55.7337499, -55.6766295, -55.5899085, -55.5607396, -55.7024884, -55.7083951, -55.7615713, -55.767487, -55.8391636, -55.8397048, -55.84086, -55.862882, -55.630918, -55.4665898]
    }

# Crear DataFrame
lugares_pago_df = pd.DataFrame(lugares_pago)

# ___________________________________________________________________________

# Añadir lugares empresa **Codigo 200.000 a 200032**
# Datos de ubicaciones fijas
ubi_empresa = {
        'CODIGO': [100001, 100002, 100003, 100004, 100005,100000,],
        'NOMCLI': ['CASA DARIO', 'CASA HUGO', 'CASA CONRADO', 'CASA MIGUEL', 'CASA MARTIN','EMPRESA'],
        'LATITUD': [-34.770962, -34.76416, -34.766187, -34.787831, -34.761527,-34.771148],
        'LONGITUD': [-55.742166, -55.86347, -55.780495, -55.850621, -55.74736, -55.758]
    }
df_ubi_empresa = pd.DataFrame(ubi_empresa)


In [4]:
# Guardamos
# Dejamos comentado hasta descubrir como cargar estos set via lectura del archivo
# el problema actual es que si simplemente importamos este archivo guardado a la hora de crear el mapa, no se guarda la estructura del dataframe resultando en error
# por eso simplemente lo declaramos en cada mapa antes de mapear, pero por las dudas
# lo dejamos aca listo para descomentar:  

# Descomentar a futuro: 

#lugares_pago_df.to_excel('lugares_pago.xlsx', index=False)
#df_ubi_empresa.to_excel('ubi_empresa.xlsx', index=False)

### 2_ Realizamos ajustes y limpiezas

### 1. Reporte de viaje (safetrack)
#### (estacionados_camion a ubi_cliente)

- Eliminamos columnas innecesarias
- Convertimos tipos de datos
- Dejamos nuevo grupo donde ["Estado_de_viaje"] == "Estacionamiento"]

In [5]:
# Eliminar las columnas innecesarias y renombrar las columnas
track = track.iloc[3:].reset_index(drop=True)
track.columns = [
'Indice', 'Numero_de_placa', 'Estado_de_viaje', 'Tiempo_de_Inicio', 
'Tiempo_Final', 'Kilometraje_km', 'Duracion', 'Lugar_de_inicio', 'Fin_Localizacion'
]

# Convertimos columnas 
track['Tiempo_de_Inicio'] = pd.to_datetime(track['Tiempo_de_Inicio'], errors='coerce')
track['Tiempo_Final'] = pd.to_datetime(track['Tiempo_Final'], errors='coerce')

# Nuevo grupo estacionados_cliente
estacionados_camion = track[track["Estado_de_viaje"] == "Estacionamiento"]

# Eliminamos columnas innecesarias
estacionados_camion.drop(columns=["Fin_Localizacion"], errors='ignore', inplace=True)
estacionados_camion.drop(columns=["Kilometraje_km"], errors='ignore', inplace=True)

Conversion de coordenadas de latitud y longitud(camion_x, camion_y) 

In [6]:
estacionados_camion.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2288 entries, 1 to 4599
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Indice            2288 non-null   object        
 1   Numero_de_placa   2288 non-null   object        
 2   Estado_de_viaje   2288 non-null   object        
 3   Tiempo_de_Inicio  2288 non-null   datetime64[ns]
 4   Tiempo_Final      2288 non-null   datetime64[ns]
 5   Duracion          2288 non-null   object        
 6   Lugar_de_inicio   2288 non-null   object        
dtypes: datetime64[ns](2), object(5)
memory usage: 143.0+ KB


In [7]:
# Asegurarse de que todos los valores en 'Lugar_de_inicio' son cadenas
estacionados_camion['Lugar_de_inicio'] = estacionados_camion['Lugar_de_inicio'].astype(str)

# Separar la columna en dos nuevas columnas, manejando posibles errores
def split_coords(lugar):
     try:
          lat, lon = lugar.split(',')
          return lat, lon
     except ValueError:
          return None, None
     
# Crear nuevas columnas 'latitud' y 'longitud' usando .loc para evitar el SettingWithCopyWarning
estacionados_camion[['latitud', 'longitud']] = estacionados_camion['Lugar_de_inicio'].apply(lambda x: pd.Series(split_coords(x)))

# Convertir las coordenadas de 'latitud' y 'longitud' a flotantes
def convert_coord(coord):
     if 'S' in coord or 'W' in coord:
          return -float(coord[:-1])
     elif 'N' in coord or 'E' in coord:
          return float(coord[:-1])
     else:
          return None
# Cambiar la columna latitud por camion_x y longitud por camion_y
estacionados_camion['latitud'] = estacionados_camion['latitud'].apply(convert_coord)
estacionados_camion['longitud'] = estacionados_camion['longitud'].apply(convert_coord)

#Eliminar filas con valores nulos en latitud y longitud
estacionados_camion = estacionados_camion.dropna(subset=['longitud', 'latitud'])


# Cambiar la columna latitud por camion_x y longitud por camion_y
estacionados_camion = estacionados_camion.rename(columns={'latitud': 'camion_x', 'longitud': 'camion_y'})

estacionados_camion = estacionados_camion.dropna(subset=['camion_x', 'camion_y', 'Numero_de_placa', 'Tiempo_de_Inicio', 'Tiempo_Final', 'Duracion'])

Verificación de coordenadas geográficas

In [8]:
# Filtrar filas con latitud fuera de rango
estacionados_camion = estacionados_camion[
    (estacionados_camion['camion_y'].between(-180, 180)) & 
    (estacionados_camion['camion_x'].between(-90, 90))
]

Duracion de objeto a float 

In [9]:
# Función para convertir "XhYMinutosZs" a formato decimal
def convertir_a_decimal(duracion):
        match = re.match(r'(?:(\d+)h)?(\d+)Minutos(\d+)s', duracion)
        if match:
            horas = int(match.group(1)) if match.group(1) else 0
            minutos = int(match.group(2))
            segundos = int(match.group(3))
            total_minutos = horas * 60 + minutos
            return total_minutos + segundos / 100  # Formato como X.YY
        return None
# Aplicar la función a la columna 'Duracion'
estacionados_camion['Duracion'] = estacionados_camion['Duracion'].apply(convertir_a_decimal)

Ajustes en Numero_de_placa y guardamos estacionados_camion.xlsx

In [10]:
# Normalizar las patentes en el DataFrame
estacionados_camion['Numero_de_placa'] = estacionados_camion['Numero_de_placa'].str.replace(' ', '').str.upper()

Dividimos el mapa mensual en semanas y calculamos percentiles para la semana especifica 

In [11]:
# Añadir una columna de semana al DataFrame
estacionados_camion['Semana'] = estacionados_camion['Tiempo_de_Inicio'].dt.isocalendar().week
estacionados_camion['Semana_del_mes'] = (estacionados_camion['Tiempo_de_Inicio'].dt.day - 1) // 7 + 1

# Filtrar por semanas del mes
semanas_del_mes = estacionados_camion['Semana_del_mes'].unique()

for semana in semanas_del_mes:
     datos_semana = estacionados_camion[estacionados_camion['Semana'] == semana]
     # Calcular percentiles para la semana específica
     duracion_25 = datos_semana['Duracion'].quantile(0.25)
     duracion_75 = datos_semana['Duracion'].quantile(0.75)

Verificación de semanas

In [12]:
# Eliminar filas con valores nulos en 'Semana_del_mes'
estacionados_camion = estacionados_camion.dropna(subset=['Semana_del_mes'])

# Filtrar por semanas del mes
semanas_del_mes = estacionados_camion['Semana_del_mes'].unique()

In [13]:
# Guardamos estacionados_camion
estacionados_camion.to_excel("estacionados_camion.xlsx", index=False)

### 2. Ubicaciones_direcciones (ubi_cliente)

In [14]:
# Leemos el set
df_cliente = pd.read_excel("../AutoMap/Ubicaciones_direcciones.xlsx", engine="openpyxl")

# Dejamos solo las columnas de interes CODIGO, NOMCLI, LATITUD, LONGITUD, FECHA.
ubi_cliente = df_cliente[['CODIGO', 'CODIGO2', 'LATITUD', 'LONGITUD']]
ubi_cliente = ubi_cliente.rename(columns={'CODIGO2': 'NOMCLI'})

# Transformaciones

# Asegurarse de que las columnas LONGITUD y LATITUD sean numéricas
ubi_cliente['LONGITUD'] = pd.to_numeric(ubi_cliente['LONGITUD'], errors='coerce')
ubi_cliente['LATITUD'] = pd.to_numeric(ubi_cliente['LATITUD'], errors='coerce')

# Limpiezas
ubi_cliente = ubi_cliente.dropna(subset=['LATITUD', 'LONGITUD', 'CODIGO', 'NOMCLI'])
ubi_cliente = ubi_cliente.drop_duplicates(subset=['NOMCLI'])
ubi_cliente.reset_index(drop=True, inplace=True)

#Guardamos 
ubi_cliente.to_excel("ubi_cliente.xlsx", index="FALSE")

### 3_ TAREAS PROYECTOS 

In [15]:
# Tareas.xlsx
df_tareas = pd.read_excel('Tareas.xlsx', engine="openpyxl")

In [16]:
# eliminamos duplicados en codigo cliente
df_tareas = df_tareas.drop_duplicates(subset=['CODIGO'], keep='first')
df_tareas.sample(5)

,PROYECTO,TAREA,DESCRIPCION,SECCION,FECHA,PERIODICIDAD,ORDEN,RESPONSABLE,CODPROY,CODTAREA,ETIQUETA,CODIGO
318,🗞️ Visitas periodicas Martin,"41920, ALICIA BIZOSA DE MAYO (City Golf)","Marcador 108\nlat/lng: (-34.75747,-55.77451)\n...",NaN,2024-07-27T03:00:00.000Z,cada 56 dias,124.0,26987950.0,2.241074e+09,7.702498e+09,"🌓Tar, 🗞️Vp",41920
469,Ruta Dario Lunes,"80280,4 SOLES ( Abigail )",NaN,NaN,2024-08-26T03:00:00.000Z,CADA LUNES,25.0,26988122.0,2.236031e+09,8.009769e+09,NaN,80280
452,Ruta Dario Lunes,"80261,SALON LOCURA (Mijo)",🟡🔥///,NaN,2024-08-26T03:00:00.000Z,CADA LUNES,7.0,26988122.0,2.236031e+09,7.562501e+09,NaN,80261
623,Ruta Dario Viernes,"22344, ANA MARTINEZ ( La Estancia) tia Facundo...",🟥/// http://maps.google.com/?q=-34.76886579585...,NaN,2024-08-23T03:00:00.000Z,cada viernes,7.0,26988122.0,2.236032e+09,7.356035e+09,NaN,22344
262,🗞️ Visitas periodicas Martin,"80239,BETTY GARRIDO MI ABUELA (La Floresta)",NaN,NaN,2024-09-16T03:00:00.000Z,cada 56 dias,68.0,26987950.0,2.241074e+09,7.349302e+09,"🌓Tar, 🔎 Ctrl, 🗞️Vp",80239


In [17]:
df_tareas['PROYECTO'].unique()

array(['Clientes Especiales', '🗞️ Visitas periodicas Hugo',
       '🗞️ Visitas periodicas Martin', '🗞️ Visitas Periodicas Dario',
       'Ruta Dario Lunes', 'Ruta Dario Martes', 'Ruta Dario Miercoles',
       'Ruta Dario Jueves', 'Ruta Dario Viernes', 'Ruta Dario Sabado',
       'Ruta Martin Lunes', 'Ruta Martin Martes', 'Ruta Martin Miercoles',
       'Ruta Martin Jueves', 'Ruta Martin Viernes', 'Ruta Martin Sabado',
       'Ruta Hugo Lunes', 'Ruta Hugo Martes', 'Ruta Hugo Miercoles',
       'Ruta Hugo Jueves', 'Ruta Hugo Sabado', '💲Creditos Hugo',
       '💲Creditos Martin', '💲Creditos Dario', '💲Creditos Administracion'],
      dtype=object)

In [18]:
def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text)
                  if unicodedata.category(c) != 'Mn')


Categorias

In [19]:
def obtener_categoria(proyecto):
    # Eliminar emojis y otros caracteres especiales
    proyecto = ''.join(c for c in proyecto if c.isascii())
    # Eliminar acentos
    proyecto = strip_accents(proyecto)
    # Convertir a minúsculas
    proyecto = proyecto.lower().strip()
    
    if 'clientes especiales' in proyecto:
        return 'Clientes Especiales'
    elif 'visitas' in proyecto:
        return 'Visitas Periódicas'
    elif 'ruta' in proyecto:
        return 'Rutas'
    elif 'creditos' in proyecto:
        return 'Creditos'
    return 'Otros'

# Aplicar la función a la columna 'PROYECTO' y crear una nueva columna 'CATEGORIA'
df_tareas['CATEGORIA'] = df_tareas['PROYECTO'].apply(obtener_categoria)

In [20]:
print(df_tareas['CATEGORIA'].value_counts())

CATEGORIA
Rutas                  290
Visitas Periódicas     244
Clientes Especiales     40
Creditos                26
Name: count, dtype: int64


In [21]:
print("Categorías únicas:")
print(df_tareas['CATEGORIA'].unique())
print("\nConteo de categorías:")
print(df_tareas['CATEGORIA'].value_counts())

Categorías únicas:
['Clientes Especiales' 'Visitas Periódicas' 'Rutas' 'Creditos']

Conteo de categorías:
CATEGORIA
Rutas                  290
Visitas Periódicas     244
Clientes Especiales     40
Creditos                26
Name: count, dtype: int64


In [22]:
df_tareas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 0 to 1220
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PROYECTO      600 non-null    object 
 1   TAREA         599 non-null    object 
 2   DESCRIPCION   293 non-null    object 
 3   SECCION       117 non-null    float64
 4   FECHA         530 non-null    object 
 5   PERIODICIDAD  530 non-null    object 
 6   ORDEN         573 non-null    float64
 7   RESPONSABLE   574 non-null    float64
 8   CODPROY       573 non-null    float64
 9   CODTAREA      597 non-null    float64
 10  ETIQUETA      329 non-null    object 
 11  CODIGO        599 non-null    object 
 12  CATEGORIA     600 non-null    object 
dtypes: float64(5), object(8)
memory usage: 65.6+ KB


In [23]:
# dejar en tareas solamente CODIGO, PROYECTO, CATEGORIA


In [24]:
df_tareas.to_excel("Tareas-limpio.xlsx", index="FALSE")